In [1]:
import json
import numpy as np
import torch
from dataset_manipulation.dataloaders import DataLoaderFactory 
from models.mmdgm import MMDGM
# Import the Polymnist (PM) encoder and decoder classes
from models.encoder_decoder.polymnist import Enc as PM_enc, Dec as PM_dec
# Import the encoders/decoders of the image and text modalities of the CUB Image Caption dataset
from models.encoder_decoder.cnn_cub_image import Enc as img_enc, Dec as img_dec
from models.encoder_decoder.cnn_cub_text import Enc as txt_enc, Dec as txt_dec

In [2]:
device = 'cpu' # 'cuda'

# Data handling

In [3]:
# Define the data folder path
data_path = '../data'

datasets_names:
- polymnist_unimodal
- polymnist_multimodal
- cub_caption
- cub_image
- cub_joint

In [4]:
dlf = DataLoaderFactory(datadir=data_path, num_workers=1, pin_memory=True)

In [5]:
# Functions used to unpack the data from the different datasets dataloaders

def unpack_data_PM(data, device=device):
    data_nolabel = data[0]
    n_idxs = len(data_nolabel)
    return [data_nolabel[idx].to(device) for idx in range(n_idxs)], data[1].to(device)

def unpack_data_cubIC(data, device=device):
    return [data[0][0].to(device), data[1][0].to(device)]

# Parameters definition
we'll use argparse in the main script

In [6]:
class Params:
    def __init__(self, num_modalities, device = 'cpu'):
        self.aggregation_method = 'concatenation'
        self.latent_dim = 16
        self.num_modalities = num_modalities
        self.device = device
        self.is_gaussian = True

# Polymnist

In [7]:
# Define the paramaeters
num_modalities = 5
latent_dim = 16
batch_size = 8

In [8]:
params = Params(num_modalities)

## Model definition

In [9]:
# Lists of the different modalities encoders/decoders
encoders = [PM_enc for _ in range(num_modalities)]
decoders = [PM_dec for _ in range(num_modalities)]

In [10]:
# Multimodal generative model
mmdgm_PM = MMDGM(encoders, decoders, params) # is it optimal to store the pass the classes of the encoders and decoders?

## Dataloader

In [11]:
train, test = dlf.get_dataloader('polymnist_multimodal', batch_size)

In [12]:
data_iter = next(iter(train))

/home/quintana/github/MM_Diffusion/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
polymnist_modalities = unpack_data_PM(data_iter)

In [14]:
pm_reconstructions = mmdgm_PM(polymnist_modalities[0])

# CUB Image Caption

In [15]:
num_modalities = 2
latent_dim = 16
batch_size = 8

In [16]:
params = Params(num_modalities)

## Model definition

In [17]:
encoders = [img_enc, txt_enc]
decoders = [img_dec, txt_dec]

In [18]:
mmdgm_cub = MMDGM(encoders, decoders, params) # is it optimal to store the pass the classes of the encoders and decoders?

## Datalaoder

In [19]:
train, test = dlf.get_dataloader('cub_joint', batch_size)

In [20]:
data_iter = next(iter(train))

In [21]:
cub_modalities = unpack_data_cubIC(data_iter)

In [22]:
cub_reconstructions = mmdgm_cub(cub_modalities)